

# Five Quantum Info Experiments

_Author: J. Lykken_

_Edited for personal use by: Carlos Alvarado_

This notebook goes with Lecture One, June 23 2020, of
"Quantum Information for Particle Theorists"
at the TASI summer school [2010.02931](https://arxiv.org/abs/2010.02931)

---



### Connect to the Google Colab cloud GPU service (IGNORE)

In [ ]:
#### GOOGLE COLAB SPECIFIC ##########################
# You will need a free Google Colab account or a paid Colab Pro account
# Put this notebook in your Google Drive, then open it using Colab
#
# CODE SNIPPET TO ACCESS THE FILES IN YOUR GOOGLE DRIVE
# IT WILL ASK YOU FOR AUTHORIZATION VIA YOUR GOOGLE ACCOUNT

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"


MessageError: Error: credential propagation was unsuccessful

### Installing cirq

Executing the following cell will attempt to install cirq. On Google Colab it may then ask you to restart the runtime; do so.

In [1]:
#!pip install cirq --quiet

# Used instead this, as suggested by the AI
!pip install cirq-google
print()

!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.8 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

To verify that Cirq is installed in your environment, try to importing cirq and print out a diagram of the Sycamore chip. It should produce a 2x11 grid of qubits.

In [2]:
# Are these modules installed?

!pip show cirq
print()

!pip show cirq-google
print()

# It seems so

Name: cirq
Version: 1.4.0
Summary: A framework for creating, editing, and invoking Noisy Intermediate Scale Quantum (NISQ) circuits.
Home-page: http://github.com/quantumlib/cirq
Author: The Cirq Developers
Author-email: cirq-dev@googlegroups.com
License: Apache 2
Location: /usr/local/lib/python3.10/dist-packages
Requires: cirq-aqt, cirq-core, cirq-google, cirq-ionq, cirq-pasqal, cirq-rigetti, cirq-web
Required-by: 

Name: cirq-google
Version: 1.4.0
Summary: The Cirq module that provides tools and access to the Google Quantum Computing Service
Home-page: http://github.com/quantumlib/cirq
Author: The Cirq Developers
Author-email: cirq-dev@googlegroups.com
License: Apache 2
Location: /usr/local/lib/python3.10/dist-packages
Requires: cirq-core, google-api-core, proto-plus, protobuf
Required-by: cirq



In [3]:
# Not possible to print the Fox Tail
# Trying instead the Sycamore

import cirq
import cirq_google
print(cirq_google.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

### Import things we might need and define path to our Google Drive directory

In [4]:
# Define a googlepath to your googledrive to load/save files.
# Modify the example below as needed
googlepath = "drive/My Drive/QuantumInfo"
from cirq.circuits import InsertStrategy

import tensorflow as tf
import numpy as np
from numpy import around, log2, isreal, all, trace, conj, outer
from math import factorial, sqrt, pi
from cmath import exp
from numpy import matrix, eye, count_nonzero, around, sum, save
import scipy, sympy
from scipy import interpolate
from sympy import Symbol
import matplotlib.pyplot as plt
from matplotlib import gridspec
import gc, random, timeit
from timeit import default_timer as timer
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# **Experiment 1**: simplest single-qubit circuit in Cirq
From Cirq tutorial and Jack Hidary's book Chapt 6

In [ ]:
# My own code

# Get a qubit
# The '0' is the qubit's coordinate
# LineQubit indicates a linear circuit
# By default, qubits start as |0>
qbit = cirq.LineQubit(0)
print(type(qbit))

<class 'cirq.devices.line_qubit.LineQubit'>


In [ ]:
# Get a circuit
circuit = cirq.Circuit()
print(type(circuit))

<class 'cirq.circuits.circuit.Circuit'>


In [ ]:
# An X gate (2x2 Pauli sigma_x matrix) is attached to the circuit
# The 'qbit' indicates to place it after qbit
circuit.append( cirq.X(qbit) )

In [ ]:
# Run a simple simulation that extracts the wavefunction of this state
# With .simulate() we can check the qubits at each stage. Not true in real life!

# Initialize a simulation for our circuit
sim = cirq.Simulator()

# Simulate the circuit when the initial state is |0> or |1>
# (one of these must always be commented)
# result = sim.simulate(circuit, initial_state=0)
resultSimulate = sim.simulate(circuit, initial_state=1)

# Hrer simulate() admits an argument
# initial_state in case you don't want
# to start with a |0>
# result = sim.simulate(circuit, initial_state=1)

In [ ]:
# Once the simulator is invoked, it is possible
# to print further stages of the circuit
print(circuit)

# The X gate will act on a qubit
# whose coordinate  in the cicuit is '0'

0: ───X───


In [ ]:
# Projects the final-state vector into the Bloch sphere
printmd("\n**Bloch Sphere of the qubit in the final state:**")
# The final_state_vector returns the state vector
# resulting from acting operations on a state. The '0'
# indicates to apply this to the qubit at location '0'
state = cirq.bloch_vector_from_state_vector(resultSimulate.final_state_vector,0)

#Prints Bloch sphere coordinates of the final state
print("x: ", around(state[0], 4), " y: ", around(state[1], 4),
      " z: ", around(state[2], 4))
printmd("\n**Indeed, X acting on |0> (North Pole, +1) gives |1>, which is the South Pole (-1) of the Bloch sphere**")
printmd("\n**Had we chosen |1> as initial state, X acting on it would return |0> with (x,y,z)=(0,0,+1)**")



**Bloch Sphere of the qubit in the final state:**

x:  0.0  y:  0.0  z:  1.0



**Indeed, X acting on |0> (North Pole, +1) gives |1>, which is the South Pole (-1) of the Bloch sphere**


**Had we chosen |1> as initial state, X acting on it would return |0> with (x,y,z)=(0,0,+1)**

In [ ]:
# Add a measurement (symbol 'M') at the end of the circuit:
circuit.append(cirq.measure(qbit, key="Final state"))
# Display the circuit:
print(circuit)

0: ───X───M('Final state')───


In [ ]:
# Invoke the Cirq quantum simulator again
# to execute the circuit multiple times:
manysimulator = cirq.Simulator()

# What's the big deal with the simulator?
# That we don't prepare the system again in the |0>
# state in order to repeat the experiment!

In [ ]:
# Run the circuit several times:
# (With .run(), as opposed to .simulate(), we
# only get to see the final result in the circuit)
resultRun = manysimulator.run(circuit, repetitions=10)
# With .run() the qubits are always initialized as |0>

In [ ]:
# Print the results:
printmd("\n**Results of 10 trials:**")
print(resultRun)
printmd("\n**X acting on |0> always returns |1>**")


**Results of 10 trials:**

Final state=1111111111



**X acting on |0> always returns |1>**

# **Experiment 2**: Circuit to create a Bell state in Cirq
From Jack Hidary's book Chapt 6

In [ ]:
# Define 'qubit' as a list of two 1-qubits
# Both qubits are chosen in |0>
qubit = [cirq.LineQubit(0),cirq.LineQubit(1)]
print(qubit)
print(type(qubit))

[cirq.LineQubit(0), cirq.LineQubit(1)]
<class 'list'>


In [ ]:
# Define a cirq circuit
circuit = cirq.Circuit()

In [ ]:
# Add a Hadamard gate to qubit 0, afterwards, a CNOT gate to both
# Notice Hadamard is cirq.H and the controlled-not is cirq.CNOT
# In qubit[i], the i iterates over the qubit number
circuit.append([
    cirq.H(qubit[0]),
    cirq.CNOT(qubit[0], qubit[1])
             ])

In [ ]:
# Invoke the simulator
sim = cirq.Simulator()
result = sim.simulate(circuit)

# Printing is now possible
# Hadarmd appears as H, controlled-not appears
# as @X where @ denotes the control qubit
# After all, the nontrivial part of CNOT is the
# X gate (Pauli-x) in the 2nd qubit
print(circuit)

0: ───H───@───
          │
1: ───────X───


In [ ]:
printmd("\n**|q0> final state Bloch coordinates:**")
state = cirq.bloch_vector_from_state_vector(result.final_state_vector,0)
print("x: ", around(state[0], 4), " y: ", around(state[1], 4), " z: ", around(state[2], 4))
# Center of the Bloch sphere, this happens for entangled states


**|q0> final state Bloch coordinates:**

x:  0.0  y:  0.0  z:  0.0


In [ ]:
printmd("\n**|q1> final state Bloch coordinates:**")
state = cirq.bloch_vector_from_state_vector(result.final_state_vector,1)
print("x: ", around(state[0], 4), " y: ", around(state[1], 4), " z: ", around(state[2], 4))
# Center of the Bloch sphere, this happens for entangled states


**|q1> final state Bloch coordinates:**

x:  0.0  y:  0.0  z:  0.0


In [ ]:
# Add a measurement at the end of the circuit
# (puts a 'M' after each qubit)
circuit.append(cirq.measure(*qubit, key="Final state"))

# Display the circuit:
printmd("\n**Circuit: (Hx1) followed by CNOT**")
print(circuit)


**Circuit: (Hx1) followed by CNOT**

0: ───H───@───M('Final state')───
          │   │
1: ───────X───M──────────────────


In [ ]:
# New simulator, performs 10 measurements
simulator10 = cirq.Simulator()

# Simulate the circuit several times:
result = simulator10.run(circuit, repetitions=10)

In [ ]:
# Print the results
printmd("\n**Results:**")
print(result)
# The the 1st (2nd) list is for the 1st (2nd) qubit


**Results:**

Final state=0110110001, 0110110001


# **Experiment 3**: Circuit to SWAP two qubit states

In [ ]:
# Define two qubits and a circuit

# 'qubit' is clearly just a list of two qubits
qubit = [cirq.LineQubit(0),cirq.LineQubit(1)]
circuit = cirq.Circuit()

In [ ]:
# An H-gate prepares |0>, the first qubit, into a |+> ket
circuit.append([
    cirq.H(qubit[0])
    ])
print(circuit)

0: ───H───


In [ ]:
# Define the symbol 't'
symbol = Symbol("t")
# So far 't' is a label

In [ ]:
# Add a parameterized XPowGate to obtain what we will call
# the initial state of qubit 1:
circuit.append([
    cirq.XPowGate(exponent=symbol)(qubit[1])
    ])
print(circuit)
# Since 't' has not been chosen, X^t |q1>
# is not specified yet

0: ───H─────

1: ───X^t───


In [ ]:
# In the next step, three CNOT gates are added
# The overall effect is known as a SWAP gate
circuit.append([
    cirq.CNOT(qubit[0], qubit[1]),
    cirq.CNOT(qubit[1], qubit[0]),
    cirq.CNOT(qubit[0], qubit[1])
               ])
print(circuit)

0: ───H─────@───X───@───
            │   │   │
1: ───X^t───X───@───X───


In [ ]:
# Two measurements are made on the qubits. But not simultaneously,
# first on |q1> and later on |q0>
circuit.append(cirq.measure(qubit[1], key='q1'))
circuit.append(cirq.measure(qubit[0], key='q0'), strategy=InsertStrategy.NEW)
print(circuit)
# The InsertStrategy.NEW tells the code to append the measurement
# gate in a posterior, newly created moment (in a posterior time slice)

0: ───H─────@───X───@─────────────M('q0')───
            │   │   │
1: ───X^t───X───@───X───M('q1')─────────────


**Side note:** Refer to Section 6.2. There's a swap of states |q0> and |q1>. Then one way of performing a measurement on |q0> is to perform it on |q1> after the action of the composite SWAP gate. Old QM would say that a 2nd measurement made on |q0> will just repeat the same result due to "wavefunction collapse". In this case that does not happen, because |q0> acquired the state of |q1> upon the action of the SWAP.

In [ ]:
# Prepare a sweep list for the 't' variable
sweep = cirq.Linspace(key=symbol.name, start=0.0, stop=1.0, length=3)
print(sweep)
# For these t-values, the XPowGate becomes, respectively, the identity,
# (1+iX)/sqrt(2), and iX. Let me remind you that XPowGate is not X^t,
# but exp(iXpit/2).

Sweep:
{'t': 0.0}
{'t': 0.5}
{'t': 1.0}


In [ ]:
# Run the circuit for all values in the sweep list for 't'
sim = cirq.Simulator()
results = sim.run_sweep(circuit, sweep, repetitions=50)

In [ ]:
# Printing the results
# For each t-value there was an initial state
# for |q1> (with |q0>=|+> in all three). What is
# printed is the measurement on the two qubits

printmd("\n**Results for t = 0:**")
print(results[0])
print()

printmd("\n**Results for t = 0.5:**")
print(results[1])
print()

printmd("\n**Results for t = 1:**")
print(results[2])
print()


**Results for t = 0:**

q0=00000000000000000000000000000000000000000000000000
q1=11010110100110100100011001110101001001111011010100




**Results for t = 0.5:**

q0=01101011101011110010011111001110001010101000100011
q1=00011001111011100000011110010111001011011001101111




**Results for t = 1:**

q0=11111111111111111111111111111111111111111111111111
q1=00111111011100110000010010111010010000101111110110



**Explanation:**
- For t=0.0, the initial states are |q0>=|+> and |q1>=|0>. Yet repeated measurements (not just one) on the qubits indicate that |q0>=|0> and |q1>=|+> at the end. We know this because |q0> seems to return 0 always, while |q1> returns |0> and |1> with equal, 50% chance. The initial states of the qubits have been effectively swapped.
- For t=0.5, the initial states are |q0>=|+> and |q1>=|i> (spin-up in the y-direction), with measurements indicating at the end that |q0>=|i> and |q1>=|+>. Notice that both contain 50% of 0's and %50's of 1s, yet the bits don't coincide every time, therefore they can't be both |+>.
- For t=1.0, the initial states are |q0>=|+> and |q1>=|1>, with measurements indicating at the end that |q0>=|1> and |q1>=|+>.

# **Experiment 4**: Quantum Teleportation
Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).

In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
a Bell State Measurement of qubit 0 (the Message) and qubit 1 (Alice's entangled qubit), the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

Example adapted from the Cirq tutorials

In [5]:
# One first defines three qubits. Before anything,
# here is a list (filled with zeroes for now)
qubit=[0,0,0]

# Now replace the first 0 with a NamedQubit,
# the 'message' qubit (lists are mutable)
qubit[0]=cirq.NamedQubit('Message')

# Replace the second qubit with another one
# named Alice
qubit[1] = cirq.NamedQubit('QAlice')

# Finally, the last zero is replaced with
# a qubit for Bob
qubit[2] = cirq.NamedQubit('QBob')

print(qubit)

[cirq.NamedQubit('Message'), cirq.NamedQubit('QAlice'), cirq.NamedQubit('QBob')]


In [6]:
# Define a circuit
circuit = cirq.Circuit()

# A Bell state is created with Alice and Bob's qubits
# (as seen in Experiment #2, one applies a H and CNOT gates
# to an initial |0,0> state)
circuit.append([
    cirq.H(qubit[1]),
    cirq.CNOT(qubit[1], qubit[2])
    ])
print(circuit)

QAlice: ───H───@───
               │
QBob: ─────────X───


In [7]:
# The message qubit is set to a random state

# First get two random real numbers between
# 0 and 1. Take Message, originally in |0>,
# and apply X^random times Y^random'
ranX = random.random()
ranY = random.random()

# Append to the message qubit an X-gate to a
# random power, followed by a Y-gate to a random
# power. It certainly randomizes qubit[0]!
circuit.append([
    cirq.X(qubit[0])**ranX,
    cirq.Y(qubit[0])**ranY
    ])

In [8]:
# So far, the circuit looks like this
print(circuit)
# Notice the message gate did not participate in
# Alice and Bob's entanglement

Message: ───X^0.801───Y^0.504───

QAlice: ────H─────────@─────────
                      │
QBob: ────────────────X─────────


In [9]:
# In the next step, the operation (H*1)CNOT is
# applied to the message and Alice's qubits. It
# first does CNOT followed by a H-gate on the
# message qubit
circuit.append([
    cirq.CNOT(qubit[0],qubit[1]),
    cirq.H(qubit[0])
    ])
print(circuit)
# U rotates |beta00> to |00> etc. It is the
# inverse of Experiment #2's operation in Eq.(14),
# which went as CNOT*(H x 1). We are bringing
# |message,Alice> to the computational basis

Message: ───X^0.801───Y^0.504───@───H───
                                │
QAlice: ────H─────────@─────────X───────
                      │
QBob: ────────────────X─────────────────


In [10]:
# Next, a measurement is added on the message and
# Alice qubits
circuit.append(cirq.measure(qubit[0], qubit[1]))
print(circuit)
# After all, measurements make sense in the comp. basis

Message: ───X^0.801───Y^0.504───@───H───M───
                                │       │
QAlice: ────H─────────@─────────X───────M───
                      │
QBob: ────────────────X─────────────────────


In [ ]:
# In another operation, a CNOT on Alice and Bob is
# applied, followed by a contro-Z on the message
# and Bob's qubits
circuit.append([
    cirq.CNOT(qubit[1], qubit[2]),
    cirq.CZ(qubit[0], qubit[2])
    ])
print(circuit)

msg: ──────X^0.544───Y^0.651───@───H───M───────@───
                               │       │       │
qalice: ───H─────────@─────────X───────M───@───┼───
                     │                     │   │
qbob: ───────────────X─────────────────────X───@───


In [11]:
# Initialize the simulator
sim = cirq.Simulator()

In [12]:
# Give a name 'q0' to the message qubit
q0 = cirq.LineQubit(0)

# The actual message (a string of classical bits) is a simulation
# acting on the circuit. One acts on |q0> and |q1> with a new
# pair of random powers of the X- and Y-gates
message = sim.simulate(
    cirq.Circuit(
        [cirq.X(q0)**ranX, cirq.Y(q0)**ranY]
        )
    )

In [13]:
# For instance, in this run the initial message is
print(message)

measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: (-0.701-0.121j)|0⟩ + (0.492+0.503j)|1⟩

phase:
output vector: |⟩


In [14]:
# Here is the 'message' as a projection on the Bloch sphere
printmd("\n**Bloch Sphere of the Message qubit in the initial state:**")
expected = cirq.bloch_vector_from_state_vector(message.final_state_vector,0)
print("x: ", around(expected[0], 4), " y: ", around(expected[1], 4),
      " z: ", around(expected[2], 4))


**Bloch Sphere of the Message qubit in the initial state:**

x:  -0.8106  y:  -0.5854  z:  0.011


In [15]:
# Records the final state of the simulation.
final_results = sim.simulate(circuit)

In [16]:
# Bob's qubit afterwards
printmd("\n**Bloch Sphere of Bob's qubit in the final state:**")
teleported = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 2)
print("x: ", around(teleported[0], 4), " y: ",
    around(teleported[1], 4), " z: ", around(teleported[2], 4))


**Bloch Sphere of Bob's qubit in the final state:**

x:  0.8106  y:  0.5854  z:  0.011


In [18]:
# Finally, the message in the final state of the simulation
printmd("\n**Bloch Sphere of the Message qubit in the final state:**")
message_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 0)
print("x: ", around(message_final[0], 4), " y: ",
    around(message_final[1], 4), " z: ", around(message_final[2], 4))


**Bloch Sphere of the Message qubit in the final state:**

x:  0.0  y:  0.0  z:  -1.0


# **Experiment 5**: Quantum Teleportation without measurement


In [19]:
# Repeat Experiment #4 without the measurement gates of
# the Bell state measurement

qubit=[0,0,0]
qubit[0]=cirq.NamedQubit('Message')
qubit[1] = cirq.NamedQubit('QAlice')
qubit[2] = cirq.NamedQubit('QBob')
circuit = cirq.Circuit()
circuit.append([
    cirq.H(qubit[1]),
    cirq.CNOT(qubit[1], qubit[2])
    ])
ranX = random.random()
ranY = random.random()
circuit.append([
    cirq.X(qubit[0])**ranX,
    cirq.Y(qubit[0])**ranY
    ])
circuit.append([
    cirq.CNOT(qubit[0],qubit[1]),
    cirq.H(qubit[0])
    ])
# SKIPPED!
# circuit.append(cirq.measure(qubit[0], qubit[1]))
circuit.append([
    cirq.CNOT(qubit[1], qubit[2]),
    cirq.CZ(qubit[0], qubit[2])
    ])
sim = cirq.Simulator()
q0 = cirq.LineQubit(0)
message = sim.simulate(
    cirq.Circuit(
        [cirq.X(q0)**ranX, cirq.Y(q0)**ranY]
        )
    )
printmd("\n**Bloch Sphere of the Message qubit in the initial state:**")
expected = cirq.bloch_vector_from_state_vector(message.final_state_vector,0)
print("x: ", around(expected[0], 4), " y: ", around(expected[1], 4),
      " z: ", around(expected[2], 4))
final_results = sim.simulate(circuit)
printmd("\n**Bloch Sphere of Bob's qubit in the final state:**")
teleported = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 2)
print("x: ", around(teleported[0], 4), " y: ",
    around(teleported[1], 4), " z: ", around(teleported[2], 4))
printmd("\n**Bloch Sphere of the Message qubit in the final state:**")
message_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 0)
print("x: ", around(message_final[0], 4), " y: ",
    around(message_final[1], 4), " z: ", around(message_final[2], 4))


**Bloch Sphere of the Message qubit in the initial state:**

x:  -0.2195  y:  -0.2479  z:  0.9436



**Bloch Sphere of Bob's qubit in the final state:**

x:  -0.2195  y:  -0.2479  z:  0.9436



**Bloch Sphere of the Message qubit in the final state:**

x:  1.0  y:  0.0  z:  0.0
